# Implementing Logistic Regression
Mihir Singh

## Abstract

### Metadata Block

In [1]:
# metadata
%load_ext autoreload
%autoreload 2
from logistic import LogisticRegression, GradientDescentOptimizer

ImportError: cannot import name 'LogisticRegression' from 'logistic' (/Users/mihirsingh/Documents/Middlebury/CSCI451/blog/posts/logistic-regression/logistic.py)


## Implementing Logistic Regression


